#### QGIS -> demand 
노선별로 경로 생성
distance 데이터 넣기

In [8]:
import pandas as pd
from collections import defaultdict, deque

# ──────────────────────────────
# 1. 파일 경로
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
xlsx_path = base_path + r"\qgis_export.xlsx"
sheet_name = "LINE"
# ──────────────────────────────

df = pd.read_excel(xlsx_path, sheet_name=sheet_name)
df = df.dropna(subset=["from_node_", "to_node_id"])
df["from_node_"] = df["from_node_"].astype(int)
df["to_node_id"] = df["to_node_id"].astype(int)

print("📌 노선별 단일 경로 (방향 무시)")
for line, group in df.groupby("RLWAY_NM"):
    # 양방향 그래프 생성
    graph = defaultdict(list)
    for u, v in zip(group["from_node_"], group["to_node_id"]):
        graph[u].append(v)
        graph[v].append(u)

    # 노드 연결 상태 진단
    degree = {n: len(adj) for n, adj in graph.items()}
    endpoints = [n for n, d in degree.items() if d == 1]
    if len(endpoints) != 2:
        print(f"⚠️ {line}: 선형 경로 아님 (끝 노드 {len(endpoints)}개)")
        continue

    # 한쪽 끝에서부터 BFS로 경로 복원
    start = endpoints[0]
    visited = set()
    path = []

    def dfs(u):
        visited.add(u)
        path.append(u)
        for v in graph[u]:
            if v not in visited:
                dfs(v)

    dfs(start)
    print(f"{line}: {path}")


📌 노선별 단일 경로 (방향 무시)
KTX강릉선: [22, 23, 34, 51]
경부고속철도: [4, 3, 1, 2, 13, 18, 28, 42, 60, 72, 75, 79]
경부선: [79, 77, 76, 73, 70, 63, 60, 59, 46, 41, 28, 17, 14, 9, 7, 1, 4]
경북선: [41, 38, 36, 40, 43]
경전선: [48, 57, 64, 65, 62, 66, 71, 73]
동해선: [55, 58, 69, 74, 72]
영동선: [51, 55, 54, 50, 43]
장항선: [14, 11, 10, 15, 16, 27, 30]
전라선: [68, 62, 56, 47, 45, 33, 30]
중앙선: [3, 5, 6, 8, 12, 22, 29, 31, 35, 39, 43, 49, 52, 67, 72, 78, 79]
충북선: [17, 18, 19, 20, 26, 29]
호남고속철도: [48, 37, 30, 21, 18, 13, 2, 1, 3]
호남선: [61, 53, 48, 44, 37, 32, 30, 24, 25, 28]


#### EDGE timestep 데이터 만들기 위해
- distance로 이동시간
- 이동시간 기반 time step 계산
- EDGE 데이터 .json 파일로 저장

In [1]:
import pandas as pd

# ────────────── 설정 ──────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_file  = base_path + r"\qgis_export.xlsx"
output_file = base_path + r"\qgis_export_with_time.xlsx"
sheet_name  = "EDGE"
# ────────────────────────────────

# 1. 원본 엑셀 전체 읽기 (모든 시트 보존 목적)
xlsx_all = pd.read_excel(input_file, sheet_name=None)

# 2. EDGE 시트만 수정
df_edge = xlsx_all[sheet_name]

# 3. time_step 계산
if 'distance' not in df_edge.columns:
    raise ValueError("'distance' 열이 EDGE 시트에 없습니다.")
df_edge['time_step'] = df_edge['distance'] / 150 * 60

# 4. 수정한 EDGE 시트 다시 넣기
xlsx_all[sheet_name] = df_edge

# 5. 전체 시트를 새 파일로 저장
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet, df in xlsx_all.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

print(f"✅ 완료: time_step 열이 추가된 파일이 저장됨 →\n{output_file}")


✅ 완료: time_step 열이 추가된 파일이 저장됨 →
D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_with_time.xlsx


In [2]:
import pandas as pd
import math

# ────────────── 설정 ──────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_file  = base_path + r"\qgis_export_with_time.xlsx"
output_file = base_path + r"\qgis_export_with_step.xlsx"
sheet_name  = "EDGE"
# ────────────────────────────────

# 1. 모든 시트 읽기
xlsx_all = pd.read_excel(input_file, sheet_name=None)

# 2. EDGE 시트 수정
df_edge = xlsx_all[sheet_name]

# 3. time_step 계산 (5로 나누고 올림)
if 'time(min)' not in df_edge.columns:
    raise ValueError("'time(min)' 열이 EDGE 시트에 없습니다.")

df_edge['time_step'] = df_edge['time(min)'].apply(lambda x: math.ceil(x / 5))

# 4. 덮어쓰기
xlsx_all[sheet_name] = df_edge

# 5. 새 파일로 저장
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet, df in xlsx_all.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

print(f"✅ 완료: 'time_step' 계산 후 저장됨 →\n{output_file}")


✅ 완료: 'time_step' 계산 후 저장됨 →
D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_with_step.xlsx


In [5]:
import pandas as pd
import json
import os
from collections import OrderedDict

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_file  = os.path.join(base_path, "qgis_export_with_step.xlsx")
sheet_name  = "EDGE"
output_json = os.path.join(base_path, "edges.json")
# ───────────────────────────────────

# 1. 데이터 불러오기
df = pd.read_excel(input_file, sheet_name=sheet_name)

# 2. 필요한 열 확인
required_cols = ['edge_id', 'from_node_id', 'to_node_id', 'time_step']
missing = set(required_cols) - set(df.columns)
if missing:
    raise ValueError(f"다음 열이 누락되었습니다: {missing}")

# 3. 정수형 edge_id 기준으로 정렬
df = df.dropna(subset=['edge_id'])
df['edge_id'] = df['edge_id'].astype(int)
df = df.sort_values(by='edge_id')

# 4. edge 딕셔너리 생성 (OrderedDict로 순서 유지)
edges = OrderedDict()

for _, row in df.iterrows():
    eid = f"e{int(row['edge_id'])}"
    n_from = f"n{int(row['from_node_id'])}"
    n_to   = f"n{int(row['to_node_id'])}"
    tstep  = int(row['time_step'])

    edges[eid] = (n_from, n_to, tstep)

# 5. JSON 저장
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(edges, f, indent=4)

print(f"✅ 정렬된 edges.json 저장 완료 →\n{output_json}")


✅ 정렬된 edges.json 저장 완료 →
D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\edges.json


In [6]:
import pandas as pd
import os
import json

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
xlsx_file = os.path.join(base_path, "qgis_export_with_step.xlsx")
sheet_name = "TRAIN"
output_json = os.path.join(base_path, "routes_nodes.json")
# ───────────────────────────────────

# 1. TRAIN 시트 읽기
df = pd.read_excel(xlsx_file, sheet_name=sheet_name)

# 2. routes_nodes 딕셔너리 생성
routes_nodes = {}

for _, row in df.iterrows():
    name = str(row["RLWAY_NM"]).strip()
    nodes_str = str(row["NODE"]).strip()

    # 노드 문자열 파싱 → ['n4', 'n3', 'n1', ...]
    node_list = [f"n{int(n)}" for n in nodes_str.split("-") if n.isdigit()]

    # 정방향 및 역방향 저장
    routes_nodes[f"{name}1"] = node_list
    routes_nodes[f"{name}2"] = node_list[::-1]

# 3. JSON 저장
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(routes_nodes, f, indent=4, ensure_ascii=False)

print(f"✅ 저장 완료: {output_json}")


✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\routes_nodes.json


In [7]:
import json
import os
from collections import OrderedDict

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_json = os.path.join(base_path, "edges.json")
output_json = os.path.join(base_path, "edges_bidirectional.json")
# ───────────────────────────────────

# 1. 기존 edges.json 불러오기
with open(input_json, "r", encoding="utf-8") as f:
    edges = json.load(f)

# 2. 양방향 딕셔너리 생성 (순서 보존)
edges_bidir = OrderedDict()

for eid, (n1, n2, t) in edges.items():
    # 정방향 edge 추가
    edges_bidir[eid] = [n1, n2, t]
    
    # 역방향 edge 추가
    eid_r = f"{eid}r"
    edges_bidir[eid_r] = [n2, n1, t]

# 3. 새 JSON 파일로 저장
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(edges_bidir, f, indent=4)

print(f"✅ 양방향 edges 저장 완료: {output_json}")


✅ 양방향 edges 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\edges_bidirectional.json


In [8]:
import json
import os
from collections import defaultdict

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
routes_json = os.path.join(base_path, "routes_nodes.json")
output_demand_json = os.path.join(base_path, "demand_template.json")
# ───────────────────────────────────

# 1. routes_nodes.json 불러오기
with open(routes_json, "r", encoding="utf-8") as f:
    routes_nodes = json.load(f)

# 2. demand 생성
demand = {}

for tr, path in routes_nodes.items():
    od_list = []
    for i in range(len(path)-1):
        for j in range(i+1, len(path)):
            o, d = path[i], path[j]
            od_list.append((o, d, None))  # 수요는 아직 없음
    demand[tr] = od_list

# 3. 저장
with open(output_demand_json, "w", encoding="utf-8") as f:
    json.dump(demand, f, indent=4, ensure_ascii=False)

print(f"✅ demand_template.json 저장 완료 →\n{output_demand_json}")


✅ demand_template.json 저장 완료 →
D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\demand_template.json


In [10]:
import json
import os

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_file  = os.path.join(base_path, "demand_template.json")
output_file = os.path.join(base_path, "demand_template_compact.json")
# ───────────────────────────────────

# 1. 불러오기
with open(input_file, "r", encoding="utf-8") as f:
    demand = json.load(f)

# 2. 저장 (수동 문자열 포맷)
with open(output_file, "w", encoding="utf-8") as f:
    f.write("{\n")
    for i, (k, od_list) in enumerate(demand.items()):
        f.write(f'    "{k}": [\n')
        line = "        "  # 들여쓰기 시작
        for j, od in enumerate(od_list):
            # JSON 값으로 인코딩 (ensure null/quote 등 맞춤)
            od_json = json.dumps(od, ensure_ascii=False)
            line += od_json
            if j < len(od_list) - 1:
                line += ", "
            if len(line) > 120:
                f.write(line + "\n")
                line = "        "
        if line.strip():
            f.write(line + "\n")
        f.write("    ]")
        if i < len(demand) - 1:
            f.write(",\n")
        else:
            f.write("\n")
    f.write("}\n")

print(f"✅ 성공적으로 저장됨 → {output_file}")


✅ 성공적으로 저장됨 → D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\demand_template_compact.json


#### 역사별 승하차데이터로 임시 journeys demand data 만들기

In [28]:
import pandas as pd
from pathlib import Path

# ─────────────── 경로 설정 ───────────────
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
qgis_path = base_dir / "qgis_export_with_step.xlsx"
ridership_path = base_dir / "station_ridership.xlsx"
# ────────────────────────────────────────

# 1. QGIS NODE 데이터에서 RLNODE_NM 추출
qgis_df = pd.read_excel(qgis_path, sheet_name="NODE")
qgis_nodes = qgis_df["RLNODE_NM"].astype(str).str.strip()

# 2. 승하차 데이터에서 RLNODE_NM 추출 (E열)
ridership_df = pd.read_excel(ridership_path)
ridership_nodes = ridership_df["RLNODE_NM"].astype(str).str.strip()

# 3. 비교
matched = sorted(set(qgis_nodes) & set(ridership_nodes))
only_in_qgis = sorted(set(qgis_nodes) - set(ridership_nodes))

# 4. 출력
print("📌 둘 다 포함된 역 (matched):")
for name in matched:
    print(f" - {name}")

print("\n📌 QGIS에는 있으나 승하차 데이터에는 없는 역 (only_in_qgis):")
for name in only_in_qgis:
    print(f" - {name}")


📌 둘 다 포함된 역 (matched):
 - 경산역
 - 계룡역
 - 곡성역
 - 공주역
 - 광명역
 - 광주송정역
 - 광천역
 - 구례구역
 - 구미역
 - 구포역
 - 기장역
 - 김제역
 - 김천구미역
 - 김천역
 - 나주역
 - 남원역
 - 논산역
 - 능주역
 - 단양역
 - 대구역
 - 대전역
 - 대천역
 - 덕소역
 - 동대구역
 - 동해역
 - 마산역
 - 목포역
 - 물금역
 - 밀양역
 - 벌교역
 - 보성역
 - 봉양역
 - 부산역
 - 삼랑진역
 - 상봉역
 - 상주역
 - 서울역
 - 서창역
 - 수원역
 - 순천역
 - 안동역
 - 양평역
 - 여수엑스포역
 - 영덕역
 - 영등포역
 - 영주역
 - 영천역
 - 예산역
 - 예천역
 - 오송역
 - 온양온천역
 - 용산역
 - 울산역
 - 원주역
 - 의성역
 - 익산역
 - 장성역
 - 장항역
 - 전주역
 - 점촌역
 - 정동진역
 - 정읍역
 - 제천역
 - 진주역
 - 천안역
 - 청량리역
 - 청주역
 - 춘양역
 - 충주역
 - 태화강역
 - 평창역
 - 평택역
 - 포항역
 - 풍기역
 - 횡성역

📌 QGIS에는 있으나 승하차 데이터에는 없는 역 (only_in_qgis):
 - 백산역
 - 보천역
 - 삼척역
 - 신경주역
 - 천안아산역(온양온천)


In [ ]:
import pandas as pd
from pathlib import Path

# ─────────────── 경로 설정 ───────────────
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
xlsx_path = base_dir / "qgis_export_with_step.xlsx"
output_path = base_dir / "qgis_export_with_ridership.xlsx"
# ────────────────────────────────────────

# 1. NODE 시트 불러오기
node_df = pd.read_excel(xlsx_path, sheet_name="NODE")

# 2. BOARD,ALIGHT 시트 불러오기
ridership_df = pd.read_excel(xlsx_path, sheet_name="BOARD,ALIGHT")

# 3. RLNODE_NM 기준 병합
merged_df = node_df.merge(
    ridership_df[["RLNODE_NM", "board_1d", "alight_1d"]],
    on="RLNODE_NM",
    how="left"
)

# 4. 새로운 파일로 저장
merged_df.to_excel(output_path, index=False)

print(f"✔️ 저장 완료: {output_path}")

✔️ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_with_ridership.xlsx


In [34]:
import pandas as pd
from pathlib import Path

# ───────────── 경로 설정 ─────────────
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
xlsx_path = base_dir / "qgis_export.xlsx"
output_path = base_dir / "qgis_export_with_timestep.xlsx"
# ─────────────────────────────────────

# 1. 엣지 정보 읽기
edge_df = pd.read_excel(xlsx_path, sheet_name="EDGE")
edge_df["from_node_id"] = edge_df["from_node_id"].astype(str).str.lower()
edge_df["to_node_id"] = edge_df["to_node_id"].astype(str).str.lower()

# 2. demand 탭 읽기
demand_df = pd.read_excel(xlsx_path, sheet_name="DEMAND")

# 3. Route 기반 Timestep 계산 함수
def compute_total_timestep(route_str):
    if pd.isna(route_str):
        return None
    nodes = route_str.strip().split("-")
    total_time = 0
    for i in range(len(nodes) - 1):
        from_n = nodes[i].replace("n", "")
        to_n = nodes[i+1].replace("n", "")

        # 해당 from↔to에 해당하는 time_step 찾기
        match = edge_df[
            ((edge_df["from_node_id"] == from_n) & (edge_df["to_node_id"] == to_n)) |
            ((edge_df["from_node_id"] == to_n) & (edge_df["to_node_id"] == from_n))
        ]

        if match.empty:
            return None  # 해당 구간 정보 없음
        total_time += match.iloc[0]["time_step"]
    return total_time

# 4. Timestep 계산 및 열 추가
demand_df["Timestep"] = demand_df["Route"].apply(compute_total_timestep)

# 5. 저장
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    demand_df.to_excel(writer, sheet_name="DEMAND", index=False)

print(f"✔️ 저장 완료: {output_path}")


✔️ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_with_timestep.xlsx


In [33]:
import pandas as pd
from pathlib import Path

# 파일 경로
base_path = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
file_path = base_path / "qgis_export.xlsx"

# 엑셀 파일 읽기
train_df = pd.read_excel(file_path, sheet_name="TRAIN")
demand_df = pd.read_excel(file_path, sheet_name="DEMAND")

# 노선 이름 → 노드 경로 dict (n 접두어 붙이기)
line_routes = {}
for _, row in train_df.iterrows():
    line_name = row["RLWAY_NM"].strip()
    node_seq = [f"n{n.strip()}" for n in str(row["NODE"]).split("-")]
    line_routes[line_name] = node_seq

# 노선 이름에서 숫자 제거해서 base name으로 매핑
def get_base_line(line):
    return ''.join(filter(lambda x: not x.isdigit(), line)).strip()

# OD에 대해 Route 생성 함수
def extract_route(line, origin, destination):
    base_line = get_base_line(line)
    nodes = line_routes.get(base_line)
    if not nodes:
        return None
    if origin not in nodes or destination not in nodes:
        return None

    try:
        idx_o = nodes.index(origin)
        idx_d = nodes.index(destination)
        # 정방향 또는 역방향 슬라이싱
        if idx_o <= idx_d:
            path = nodes[idx_o:idx_d + 1]
        else:
            path = nodes[idx_o:idx_d - 1:-1]  # 역방향
        return "-".join(path)
    except ValueError:
        return None


# Route 열 추가
demand_df["Route"] = demand_df.apply(
    lambda row: extract_route(row["Line"], row["Origin"], row["Destination"]),
    axis=1
)

# 저장
output_path = base_path / "demand_with_route_filled.xlsx"
demand_df.to_excel(output_path, index=False)
print(f"✅ 저장 완료: {output_path}")


✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\demand_with_route_filled.xlsx


In [37]:
import pandas as pd
from pathlib import Path

# ───────────── 경로 설정 ─────────────
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
xlsx_path = base_dir / "qgis_export.xlsx"
output_path = base_dir / "qgis_export_with_node_id.xlsx"
# ─────────────────────────────────────

# 1. NODE 시트 읽기 (참조용)
node_df = pd.read_excel(xlsx_path, sheet_name="NODE")
node_df = node_df[["RLNODE_NM", "node_id"]].dropna()
node_df["RLNODE_NM"] = node_df["RLNODE_NM"].astype(str).str.strip()

# 2. BOARD_ALIGHT 시트에 node_id 매칭
df = pd.read_excel(xlsx_path, sheet_name="BOARD_ALIGHT")
df["RLNODE_NM"] = df["RLNODE_NM"].astype(str).str.strip()

# 매칭 수행
merged_df = df.merge(node_df, on="RLNODE_NM", how="left")

# 3. 저장
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    merged_df.to_excel(writer, sheet_name="BOARD_ALIGHT", index=False)

print(f"✔️ 저장 완료: {output_path}")


✔️ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_with_node_id.xlsx


#### Journeys data 추가

In [47]:
import pandas as pd
import numpy as np
from pathlib import Path

# ────── CONFIG ────────────────────────────────────────────────────────────
file_path   = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export.xlsx")
output_path = file_path.with_stem(file_path.stem + "_with_journeys")

min_board   = 1      # 승차가 0/누락인 역에 부여할 최소 승차 인원
min_alight  = 1      # 하차가 0/누락인 역에 부여할 최소 하차 인원
epsilon_od  = 1      # 유효 OD(승·하차>0)에 심을 최소 통행량
max_iter    = 50
eps_conv    = 1e-6
# ──────────────────────────────────────────────────────────────────────────

# 1. 데이터 읽기 -----------------------------------------------------------
bal    = pd.read_excel(
            file_path, sheet_name="BOARD_ALIGHT",
            usecols=["node_id", "board_1d", "alight_1d"]
        )
demand = pd.read_excel(file_path, sheet_name="DEMAND")

# 1‑a. Origin / Destination ⇒ 정수 node_id
demand["O_id"] = demand["Origin"].str.lstrip("n").astype(int)
demand["D_id"] = demand["Destination"].str.lstrip("n").astype(int)

# 2. BOARD_ALIGHT 누락/중복 처리 ------------------------------------------
#   2‑a. 중복 node_id 합산
bal_agg = (bal.groupby("node_id", as_index=False)
               .agg(board=("board_1d", "sum"),
                    alight=("alight_1d", "sum")))

#   2‑b. DEMAND 에 등장하지만 bal 에 없는 노드 추가
all_nodes = pd.unique(demand[["O_id", "D_id"]].values.ravel())
bal_full  = pd.DataFrame({"node_id": all_nodes}).merge(
                bal_agg, on="node_id", how="left"
            )

#   2‑c. 승·하차 0/NaN → 최소값 주입
bal_full["board"]  = pd.to_numeric(bal_full["board"],  errors="coerce").fillna(0)
bal_full["alight"] = pd.to_numeric(bal_full["alight"], errors="coerce").fillna(0)

bal_full.loc[bal_full["board"]  == 0, "board"]  = min_board
bal_full.loc[bal_full["alight"] == 0, "alight"] = min_alight

# 3. 총 승·하차량 매핑 ------------------------------------------------------
board_map  = bal_full.set_index("node_id")["board"].to_dict()
alight_map = bal_full.set_index("node_id")["alight"].to_dict()

demand["o_total"] = demand["O_id"].map(board_map)
demand["d_total"] = demand["D_id"].map(alight_map)

# 4. 초기 OD 행렬 + ε 주입 ---------------------------------------------------
f = 1.0 / demand["Timestep"].replace(0, np.nan)
demand["T"] = (demand["o_total"] * demand["d_total"] * f).fillna(0.0)

valid_od = (demand["o_total"] > 0) & (demand["d_total"] > 0)
demand.loc[valid_od, "T"] += epsilon_od      # 최소 통행량 심기

# 5. IPF (Iterative Proportional Fitting) ----------------------------------
for _ in range(max_iter):
    row_sum = demand.groupby("O_id")["T"].transform("sum").replace(0, np.nan)
    demand["T"] *= demand["o_total"] / row_sum

    col_sum = demand.groupby("D_id")["T"].transform("sum").replace(0, np.nan)
    demand["T"] *= demand["d_total"] / col_sum

    if max((row_sum - demand["o_total"]).abs().max(),
           (col_sum - demand["d_total"]).abs().max()) < eps_conv:
        break

# 6. journeys 계산 및 후처리 -------------------------------------------------
demand["journeys"] = demand["T"].round(0).astype(int)   # 정수(명)로 반올림
out = demand.drop(columns=["O_id", "D_id", "o_total", "d_total", "T"])

# 7. 결과 저장 --------------------------------------------------------------
if output_path.exists():
    mode, sheet_opt = "a", "overlay"
else:
    mode, sheet_opt = "w", None                 # 새 파일 — 시트 존재 안 하므로 옵션 불필요

with pd.ExcelWriter(output_path,
                    engine="openpyxl",
                    mode=mode,
                    if_sheet_exists=sheet_opt) as wr:
    out.to_excel(wr, sheet_name="DEMAND", index=False)

print("✅ journeys 열이 채워진 파일:", output_path)


✅ journeys 열이 채워진 파일: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_with_journeys.xlsx


In [48]:
import pandas as pd
from pathlib import Path

# 파일 경로
file_path = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export.xlsx")
output_path = file_path.with_stem(file_path.stem + "_journeys_fixed")

# DEMAND 시트 읽기
demand = pd.read_excel(file_path, sheet_name="DEMAND")

# journeys 열에서 0인 값을 1로 변경
demand["journeys"] = demand["journeys"].apply(lambda x: 1 if x == 0 else x)

# 새 파일로 저장
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    demand.to_excel(writer, sheet_name="DEMAND", index=False)

print(f"✅ Modified file saved to: {output_path}")


✅ Modified file saved to: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_journeys_fixed.xlsx


In [56]:
import pandas as pd
from collections import defaultdict
from pathlib import Path

# ────────── 1. 입력 경로 및 시트 읽기 ──────────
file_path = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export.xlsx")
df = pd.read_excel(file_path, sheet_name="DEMAND",
                   usecols=["Line", "Route", "journeys"])

# ────────── 2. Edge‑별 탑승 인원 누적 ──────────
edge_load = defaultdict(int)           # {(Line, from, to): 총탑승}

for line, route, pax in df.itertuples(index=False):
    nodes = route.split("-")           # ex) ["n4","n3","n1",...]
    for frm, to in zip(nodes[:-1], nodes[1:]):
        edge_load[(line, frm, to)] += pax   # 방향 구분 O (필요 없으면 정렬해서 key 통일)

# DataFrame 화
edge_df = (pd.DataFrame([(l,f,t,c) for (l,f,t),c in edge_load.items()],
                        columns=["Line","From","To","Load"]))

# ────────── 3. 노선별 최대 탑승 인원 계산 ──────────
max_by_line = (edge_df.groupby("Line")["Load"]
                        .agg(MaxLoad="max")
                        .reset_index())

# (선택) 어느 Edge 에서 최대가 나왔는지도 보고 싶다면:
idx = edge_df.groupby("Line")["Load"].idxmax()
edge_peak = edge_df.loc[idx].reset_index(drop=True)   # Line, From, To, Load

# ────────── 4. 결과 저장 or 출력 ──────────
print("\n===== 노선별 최대 탑승 인원 =====")
print(max_by_line.to_string(index=False))

print("\n===== 최대 탑승 Edge 상세(선택) =====")
print(edge_peak.to_string(index=False))



===== 노선별 최대 탑승 인원 =====
   Line  MaxLoad
KTX강릉선1      388
KTX강릉선2      426
경부고속철도1    29781
경부고속철도2    29708
   경부선1    24630
   경부선2    24591
   경북선1      113
   경북선2      113
   경전선1     1655
   경전선2     1660
   동해선1     1966
   동해선2     2113
   영동선1       61
   영동선2       66
   장항선1     1187
   장항선2     1184
   전라선1     2868
   전라선2     2836
   중앙선1     5046
   중앙선2     4975
   충북선1     1191
   충북선2     1187
호남고속철도1     3031
호남고속철도2     3106
   호남선1     3479
   호남선2     3483

===== 최대 탑승 Edge 상세(선택) =====
   Line From  To  Load
KTX강릉선1  n23 n22   388
KTX강릉선2  n22 n23   426
경부고속철도1   n4  n3 29781
경부고속철도2   n3  n4 29708
   경부선1   n4  n1 24630
   경부선2   n1  n4 24591
   경북선1  n38 n41   113
   경북선2  n38 n36   113
   경전선1  n66 n71  1655
   경전선2  n71 n66  1660
   동해선1  n69 n74  1966
   동해선2  n72 n74  2113
   영동선1  n55 n54    61
   영동선2  n54 n55    66
   장항선1  n15 n16  1187
   장항선2  n16 n15  1184
   전라선1  n33 n45  2868
   전라선2  n45 n33  2836
   중앙선1   n3  n5  5046
   중앙선2   n5  n3  4975
 

In [60]:
import pandas as pd, json, math
from collections import defaultdict
from pathlib import Path

# ──────────────────────── 경로 설정 ────────────────────────
BASE     = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
xlsx_fp  = BASE / "qgis_export.xlsx"
json_in  = BASE / "json" / "demand.json"
json_out = BASE / "json" / "demand_split.json"

# ───────────────── 1. DEMAND 시트 → Edge 최대 탑승 ─────────────────
df = pd.read_excel(xlsx_fp, sheet_name="DEMAND",
                   usecols=["Line", "Route", "journeys"])

edge_load = defaultdict(int)          # {(Line, u, v): 승객 누적}

for line, route, pax in df.itertuples(index=False):
    nodes = route.split("-")
    for u, v in zip(nodes[:-1], nodes[1:]):
        edge_load[(line, u, v)] += pax

edge_df = (pd.DataFrame([(l,u,v,c) for (l,u,v),c in edge_load.items()],
                        columns=["Line","From","To","Load"]))

max_load = (edge_df.groupby("Line")["Load"]
                     .max()
                     .to_dict())      # {Line: 최대 Edge 탑승}

# ───────────────── 2. 노선별 열차 1편성 최대 좌석수 매핑 ────────────────
def train_capacity(line: str) -> int:
    """노선 이름을 받아 보수적 1편성 최대 좌석수를 반환"""
    if line.startswith("KTX강릉선"):
        return 381           # KTX‑Eum
    if line.startswith("경부고속철도"):
        return 515           # KTX‑청룡 / 산천
    # 그 외 일반선·호남고속철도 등 → 산천 보수치
    return 410               # KTX‑Sancheon

cap_map   = {ln: train_capacity(ln) for ln in max_load}
train_cnt = {ln: max(1, math.ceil(max_load[ln] / cap_map[ln]))
             for ln in max_load}

print("\n=== 노선별 최대 Edge 탑승·필요 열차 대수 ===")
for ln in sorted(train_cnt):
    print(f"{ln:15s}  최대탑승 = {max_load[ln]:5.0f}명  "
          f"/  용량 = {cap_map[ln]}  →  열차 {train_cnt[ln]}대")

# ───────────────── 3. demand.json 불러와서 쪼개기 ─────────────────
with open(json_in, "r", encoding="utf-8") as f:
    demand_js = json.load(f)

split_js = {}   # 새로운 dict

for ln, od_list in demand_js.items():
    n_train = train_cnt.get(ln, 1)          # 매핑 안 되면 1대
    for idx in range(n_train):
        key = f"{ln}_{idx+1}"
        split_js[key] = []
    for o, d, j in od_list:
        share = round(j / n_train, 2)
        for idx in range(n_train):
            split_js[f"{ln}_{idx+1}"].append([o, d, share])

# ───────────────── 4. JSON 저장 (한 노선 = 한 줄) ─────────────────
with open(json_out, "w", encoding="utf-8") as f:
    f.write("{\n")
    for i, (line, trips) in enumerate(split_js.items()):
        line_str = json.dumps(line, ensure_ascii=False)
        trips_str = json.dumps(trips, ensure_ascii=False, separators=(",", ":"))
        comma = "," if i < len(split_js) - 1 else ""
        f.write(f"  {line_str}: {trips_str}{comma}\n")
    f.write("}\n")

print(f"\n✅ 줄바꿈 최소화된 JSON 저장 완료 → {json_out}")



=== 노선별 최대 Edge 탑승·필요 열차 대수 ===
KTX강릉선1          최대탑승 =   388명  /  용량 = 381  →  열차 2대
KTX강릉선2          최대탑승 =   426명  /  용량 = 381  →  열차 2대
경부고속철도1          최대탑승 = 29781명  /  용량 = 515  →  열차 58대
경부고속철도2          최대탑승 = 29708명  /  용량 = 515  →  열차 58대
경부선1             최대탑승 = 24630명  /  용량 = 410  →  열차 61대
경부선2             최대탑승 = 24591명  /  용량 = 410  →  열차 60대
경북선1             최대탑승 =   113명  /  용량 = 410  →  열차 1대
경북선2             최대탑승 =   113명  /  용량 = 410  →  열차 1대
경전선1             최대탑승 =  1655명  /  용량 = 410  →  열차 5대
경전선2             최대탑승 =  1660명  /  용량 = 410  →  열차 5대
동해선1             최대탑승 =  1966명  /  용량 = 410  →  열차 5대
동해선2             최대탑승 =  2113명  /  용량 = 410  →  열차 6대
영동선1             최대탑승 =    61명  /  용량 = 410  →  열차 1대
영동선2             최대탑승 =    66명  /  용량 = 410  →  열차 1대
장항선1             최대탑승 =  1187명  /  용량 = 410  →  열차 3대
장항선2             최대탑승 =  1184명  /  용량 = 410  →  열차 3대
전라선1             최대탑승 =  2868명  /  용량 = 410  →  열차 7대
전라선2             최대탑승 =  2836명  /  용량 = 410  

출발 timestep 설정

In [17]:
import pandas as pd
import math

# 파일 경로
file_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export.xlsx"
sheet_name = "EDGE"

# 엑셀 불러오기
df = pd.read_excel(file_path, sheet_name=sheet_name)

# 'time(min)' 열이 있는지 확인
if "time(min)" not in df.columns:
    raise KeyError("'time(min)' 열이 존재하지 않습니다.")

# 소숫점 올림하여 time_step_15 계산
df["time_step_15"] = df["time(min)"].apply(lambda x: math.ceil(x / 15))

# 덮어쓰기 저장
with pd.ExcelWriter(file_path, mode="a", if_sheet_exists="overlay", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ 'time_step_15' 열이 성공적으로 추가되었습니다.")


✅ 'time_step_15' 열이 성공적으로 추가되었습니다.


In [ ]:
import json
import os

# 파일 경로
base_dir = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json"
dep_path = os.path.join(base_dir, "dep_time.json")
demand_path = os.path.join(base_dir, "demand.json")
routes_path = os.path.join(base_dir, "routes_nodes.json")

# 1. dep_time.json 로드
with open(dep_path, encoding="utf-8") as f:
    dep_data = json.load(f)
valid_trains = set(dep_data.keys())

# 2. demand.json 필터링
with open(demand_path, encoding="utf-8") as f:
    demand_data = json.load(f)
filtered_demand = {k: v for k, v in demand_data.items() if k in valid_trains}

# 3. routes_nodes.json 필터링
with open(routes_path, encoding="utf-8") as f:
    routes_data = json.load(f)
filtered_routes = {k: v for k, v in routes_data.items() if k in valid_trains}

# 4. 저장 (덮어쓰기)
with open(demand_path, "w", encoding="utf-8") as f:
    json.dump(filtered_demand, f, ensure_ascii=False, indent=2)

with open(routes_path, "w", encoding="utf-8") as f:
    json.dump(filtered_routes, f, ensure_ascii=False, indent=2)

print("✅ demand.json과 routes_nodes.json이 dep_time.json 기준으로 성공적으로 필터링되었습니다.")


✅ dep_time.json trimmed and saved (cutoff by rounded threshold).


In [34]:
import json
import os

# 경로 설정
base_dir = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json"
demand_path = os.path.join(base_dir, "demand_filtered.json")
routes_path = os.path.join(base_dir, "routes_nodes_filtered.json")

def save_single_line_json(data, filepath):
    with open(filepath, "w", encoding="utf-8") as f:
        f.write("{\n")
        for i, (k, v) in enumerate(data.items()):
            line = f'  "{k}": {json.dumps(v, ensure_ascii=False)}'
            if i < len(data) - 1:
                line += ","
            f.write(line + "\n")
        f.write("}")

# 파일 불러오기
with open(demand_path, encoding="utf-8") as f:
    demand_data = json.load(f)
with open(routes_path, encoding="utf-8") as f:
    routes_data = json.load(f)

# 한 줄당 기차 하나씩 저장
save_single_line_json(demand_data, demand_path)
save_single_line_json(routes_data, routes_path)

print("✅ 줄바꿈 포맷 정리 완료.")


✅ 줄바꿈 포맷 정리 완료.


In [1]:
import json
from pathlib import Path

# ───────────────────────────────
# 1. 파일 경로 설정
file_path = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json\demand.json")
output_path = file_path.parent / "demand_filtered.json"

# ───────────────────────────────
# 2. 데이터 불러오기
with open(file_path, "r", encoding="utf-8") as f:
    demand_data = json.load(f)

# ───────────────────────────────
# 3. 필터링 및 통계
original_count = 0
filtered_count = 0

filtered_data = {}

for train_id, od_list in demand_data.items():
    original_count += len(od_list)
    
    # 수요 10 이상인 OD쌍만 남기기
    new_od_list = [od for od in od_list if od[2] >= 10]
    filtered_count += len(new_od_list)
    
    # 결과가 남아있는 경우에만 저장
    if new_od_list:
        filtered_data[train_id] = new_od_list

# ───────────────────────────────
# 4. 필터링 결과 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

# ───────────────────────────────
# 5. 결과 출력
print(f"🔹 필터링 전 OD쌍 개수: {original_count}")
print(f"🔹 필터링 후 OD쌍 개수: {filtered_count}")
print(f"🔹 제거된 OD쌍 개수: {original_count - filtered_count}")
print(f"✅ 저장 완료: {output_path}")


🔹 필터링 전 OD쌍 개수: 12426
🔹 필터링 후 OD쌍 개수: 3607
🔹 제거된 OD쌍 개수: 8819
✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json\demand_filtered.json


In [3]:
import json
from pathlib import Path

# ───────────────────────────────
# 1. 경로 설정
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json")
dep_time_path = base_dir / "dep_time.json"
routes_nodes_path = base_dir / "routes_nodes.json"
dep_time_out = base_dir / "dep_time_filtered.json"
routes_nodes_out = base_dir / "routes_nodes_filtered.json"

# ───────────────────────────────
# 2. dep_time.json 불러오기 및 필터링 (값이 13 미만인 것만)
with open(dep_time_path, "r", encoding="utf-8") as f:
    dep_time = json.load(f)

filtered_dep_time = {k: v for k, v in dep_time.items() if v < 12}
print(f"✅ dep_time 필터링: {len(dep_time)} → {len(filtered_dep_time)}개 남음")

# 저장
with open(dep_time_out, "w", encoding="utf-8") as f:
    json.dump(filtered_dep_time, f, ensure_ascii=False, indent=2)

# ───────────────────────────────
# 3. routes_nodes.json 불러오기 및 해당 키만 남기기
with open(routes_nodes_path, "r", encoding="utf-8") as f:
    routes_nodes = json.load(f)

# dep_time에서 남은 키만 유지
filtered_routes_nodes = {k: v for k, v in routes_nodes.items() if k in filtered_dep_time}
print(f"✅ routes_nodes 필터링: {len(routes_nodes)} → {len(filtered_routes_nodes)}개 남음")

# 저장
with open(routes_nodes_out, "w", encoding="utf-8") as f:
    json.dump(filtered_routes_nodes, f, ensure_ascii=False, indent=2)

# ───────────────────────────────
# 4. 결과 요약
print(f"\n📁 저장 완료:")
print(f"→ {dep_time_out.name}")
print(f"→ {routes_nodes_out.name}")


✅ dep_time 필터링: 122 → 106개 남음
✅ routes_nodes 필터링: 122 → 106개 남음

📁 저장 완료:
→ dep_time_filtered.json
→ routes_nodes_filtered.json


#### Capacity
- Demand_data이용해서 엣지마다 흐르는 demand양 계산한 파일: 6.edge_journeys_summary.xlsx
- e1 n1-n2 1502241
- e2 n2-n1 389459.5
- e1,e2,e3,e4, .. 넘버링 qgis와 다름 (양방향 고려)
- 양방향 엣지에 대해서는 서로 동일한 capacity를 갖도록 해야함

In [3]:
import pandas as pd
from pathlib import Path

# 1. 파일 불러오기
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
input_path = base_dir / "6.edge_journeys_summary.xlsx"
df = pd.read_excel(input_path)

# 2. 여유 계수 설정 (예: 1.2배)
buffer_ratio = 1.2

# 3. (u, v) / (v, u) 형태를 고려해 양방향 키 만들기
df["pair_key"] = df.apply(lambda row: tuple(sorted([row["from_node_name"], row["to_node_name"]])), axis=1)

# 4. 같은 pair_key 그룹 내에서 최대 journeys 선택
grouped = df.groupby("pair_key", as_index=False).agg({
    "journeys": "max"
})
grouped["capacity"] = grouped["journeys"] * buffer_ratio

# 5. 다시 원래 방향성 edge에 capacity 연결
df = df.merge(grouped[["pair_key", "capacity"]], on="pair_key", how="left")

# 6. 컬럼 정리 및 저장
df = df.drop(columns=["pair_key"])
output_path = base_dir / "7.edge_capacity_summary.xlsx"
df.to_excel(output_path, index=False)

print(f"Capacity summary saved → {output_path}")


Capacity summary saved → D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\7.edge_capacity_summary.xlsx


#### 노선별 승객 수요 데이터 -> 필요한 기차수 계산

In [31]:
import json, math, pandas as pd, os

# ───────────────────────────────────────────────
# 1. 파일 읽기
file_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\5.demand_data_kofull.json"
df = pd.read_json(file_path)
df["journeys_day"] = df["journeys"] / 365

# ───────────────────────────────────────────────
# 2. 파라미터
TARGET_LOAD    = 1.0          # 100 % 탑승률
TURNAROUND_MIN = 15           # 종점 버퍼(분)
HOURS_PER_DAY  = 17
DEFAULT_SEAT   = 400          # seat_map 에 없을 때 임시값

# 노선별 좌석수 (앞 글자 매칭용)
seat_map = {
    # 고속열차
    "KTX강릉선":    381,   # KTX-이음 (KTX-Eum) 381석 :contentReference[oaicite:2]{index=2}
    "KTX경부선":    955,   # KTX-I 955석(935~955) :contentReference[oaicite:3]{index=3}
    "KTX호남선":    955,
    "경부고속철도":   955,   # KTX-산천
    "경부선":         900,   # SRT 편성(예시)
    # ITX 계열
    "ITX-새마을":   376,   # ITX-새마을
    "ITX-청춘":     402,   # ITX-청춘
    # 일반열차
    "무궁화호":     920,   # 좌석+입석 허용, 편성 좌석 920 부근
}

# ───────────────────────────────────────────────
def train_stats(group):
    line = group.name
    seats = next((v for k, v in seat_map.items() if line.startswith(k)), DEFAULT_SEAT)

    total_pax = group["journeys_day"].sum()
    single_run = group["time_min"].max()
    round_trip = single_run + TURNAROUND_MIN
    cycles = max(1, (HOURS_PER_DAY * 60) // round_trip)

    cap_per_set = seats * TARGET_LOAD * cycles
    sets_needed = math.ceil(total_pax / cap_per_set) if cap_per_set else None

    return pd.Series({
        "cycles_day": cycles,
        "train_sets_needed": sets_needed
    })

# 4. 계산
result = df.groupby("line").apply(train_stats).reset_index()

print(result)


      line  cycles_day  train_sets_needed
0   KTX강릉선          24                  1
1   KTX동해선          39                  1
2      경강선          23                  2
3   경부고속철도           8                 23
4      경부선           1                 65
5      경북선          18                  1
6      경의선          46                  1
7      경인선          35                  1
8      경전선           5                  3
9     공항철도          20                  2
10     대구선          44                  1
11   동해남부선          12                  3
12     동해선          16                  1
13     삼척선          51                  1
14     영동선          12                  2
15     장항선          11                  2
16     전라선          17                  2
17   중부내륙선          36                  1
18     중앙선           2                 23
19     충북선          11                  1
20     태백선          19                  1
21  호남고속철도          20                  1
22     호남선          10            

C:\Users\Minji Kang\AppData\Local\Temp\ipykernel_38976\2930921966.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby("line").apply(train_stats).reset_index()
